# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

This notebook is part of an `nbdev` repo. The package is called `formalyzer` and I want there in the end to be a command line tool called `formalyzer`. what do i need to do to make that a command line tool available for nbdev?   i suspect i need to create a notebook titled `formalyzer.ipyb` and then somehow I need to declare it in some kind of `settings.ini` file?

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Great question! You're on the right track. With nbdev, you can create CLI tools using the `fastcore.script` module (specifically the `@call_parse` decorator). Here's what you need:

1. **Create a function** that will be your CLI entry point, decorated with `@call_parse`
2. **Add a console_scripts entry** in `settings.ini`

For the `settings.ini` part, you'd add something like:

```ini
console_scripts = formalyzer:formalyzer.main
```

This tells Python's packaging system: "create a command called `formalyzer` that runs the `main` function from the `formalyzer` module."

A few questions to help you get started:
- Are you familiar with `fastcore.script` and the `@call_parse` decorator for creating CLI functions?
- What do you want your `formalyzer` command to actually *do*? (This will help determine what arguments it needs)

```markdown
formalyzer: 

Reads PDF reccomendation letter, fills in admissions form(s)

usage: 
  formalyzer <recc_letter.pdf> <url_list.txt>

Instead of url_list.txt, a single URL can be given (esp. for testing purposes) 

Description: 
Formalyzer will scrape the text from the PDF recc letter, 
and for each URL in url_list, it will: 
- launch a browser tab for that url 
- fill in the form using what the LLM has gleaned from the recc letter
- attach the PDF via the form's upload/attachment button
...and do no more. 
The user will need to review the page and press the Submit button manually.


Requirements: 
- Playwright 
- ANTHROPIC_API_KEY env var. (Could support other LLMs layer)
- pdftotext 

Author: Scott H. Hawley, @drscotthawley
```



In [ ]:
!pwd && ls && ls -al ..

/app/data/formalyzer/nbs
00_core.ipynb  _quarto.yml  index.ipynb  nbdev.yml  styles.css
total 68
drwxrwsr-x  7 solveit solveit  4096 Nov 26 02:54 .
drwxrwsr-x 14 solveit solveit  4096 Nov 26 02:50 ..
drwxrwsr-x  8 solveit solveit  4096 Nov 26 02:57 .git
drwxrwxr-x  3 solveit solveit  4096 Sep  1  2024 .github
-rw-rw-r--  1 solveit solveit  4688 Nov 26 02:50 .gitignore
-rw-rw-r--  1 solveit solveit 11337 Sep  1  2024 LICENSE
-rw-rw-r--  1 solveit solveit   111 Sep  1  2024 MANIFEST.in
-rw-rw-r--  1 solveit solveit  1368 Nov 26 02:54 README.md
drwxrwsr-x  4 solveit solveit  4096 Nov 26 02:54 _proc
drwxrwsr-x  2 solveit solveit  4096 Nov 26 02:53 formalyzer
drwxrwsr-x  2 solveit solveit  4096 Nov 26 02:53 nbs
-rw-rw-r--  1 solveit solveit    87 Nov 26 02:53 pyproject.toml
-rw-rw-r--  1 solveit solveit  1026 Nov 26 02:53 settings.ini
-rw-rw-r--  1 solveit solveit  2891 Sep  1  2024 setup.py


This is a new nbdev repo. currently we are writing the file `formalizer/nbs/00_core.py`.  Presumably this is where I will put core utilities that could be called. should I also create a `formalizer/nbs/01_formalyzer.py`? or can I just define some function within this core notebook to serve as the main function for the formalizer script?  and no I'm not familiar with `fastcore.script`. 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Good questions! Either approach works — you can put your CLI entry point in `00_core.ipynb` or create a separate notebook like `01_cli.ipynb`. It's really a matter of organization preference.

A common pattern is:
- `00_core.ipynb` → utility functions (PDF parsing, form filling helpers, etc.)
- `01_cli.ipynb` → the CLI entry point that imports and uses core utilities

This keeps things modular, but for a small tool it's fine to have everything in one notebook too.

As for `fastcore.script` — it's quite simple! The key piece is the `@call_parse` decorator. When you decorate a function with it, the function's parameters automatically become command-line arguments. Type hints and default values control how they're parsed.

For example, a function like:

```python
@call_parse
def main(pdf_path: str, urls: str): ...
```

...would give you a CLI that accepts two positional arguments.

Would you like to:
1. First see how `@call_parse` works with a tiny example, or
2. Jump straight into sketching out the structure for your `formalyzer` CLI?

In [ ]:
#| export
import os 
from fastcore.script import call_parse

@call_parse
def main(pdf_path: str, urls: str):
    pdf_path = os.path.expanduser(pdf_path) 
    print("pdf_path = ",pdf_path)
    assert os.path.exists(pdf_path), f"File not found: {pdf_path}"
    if os.path.exists(os.path.expanduser(urls)): 
        print(f"file {urls} exists. Reading.")
    else: 
        print(f"No file {urls}. Treating it as a single url") 

In [ ]:
main("~/BrodyBlackwood.pdf","~/brody_urls.txt")

pdf_path =  /app/data/BrodyBlackwood.pdf
file ~/brody_urls.txt exists. reading...


Okay, I got a basic main thing running. Now how do I tell nbdev that I want this thing to be called whenever somebody runs Formalizer. In other words, how do I install Formalizer as a command line tool so that the Bash shell can run it? 

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠🧠

Nice work getting that running! Now you need to do two things:

**1. Export the function** — Add this directive in a cell before your `main` function:

```python
#| export
```

This tells nbdev to include it in the generated module.

**2. Edit `settings.ini`** — Open `../settings.ini` and add a line like:

```ini
console_scripts = formalyzer:formalyzer.core:main
```

The format is `command_name:module_path:function_name`. Since your `00_core.ipynb` exports to `formalyzer/core.py`, the module path is `formalyzer.core`.

After making those changes, run `nbdev_export` and then `pip install -e .` from the repo root to install it in editable mode.

Want to try those steps and let me know how it goes?